In [1]:
import os
os.chdir("..")
%pwd

'c:\\Users\\karthikeya\\Fraud_Detection'

In [35]:
import os
from dataclasses import dataclass
import sqlite3

import pandas as pd
import numpy as np

from src.logger import logger

In [78]:
@dataclass
class DataBaseConfig:
    db_path : str = r"C:\Users\karthikeya\Desktop\Database_Capstones.db"
    db_name : str = "Database_Capstones"
    sql_query : str = "SELECT *  FROM Fraud_detection"

In [81]:
from abc import ABC, abstractmethod
import sqlite3
import pandas as pd

# Step 1: Define the Abstract Product Interface
class Connector(ABC):
    @abstractmethod
    def connect(self):
        """
        Abstract method to connect to an SQL database
        """
        pass
    
    @abstractmethod
    def execute_query(self, query: str):
        """
        Abstract method to execute the query
        """
        pass

# Step 2: Implement Concrete Product (SQL Connector)
class SQLite3Connector(Connector):
    def __init__(self, db_name: str):
        self.db_name = db_name
        # self.connection = None
    
    def connect(self) -> str:
        """
        Connects to a database given in db_name string
        Returns:
            str: Connection success message
        """
        self.connection = sqlite3.connect(self.db_name)
        return f"Connected to SQL database: {self.db_name}"
    
    def execute_query(self, query: str) -> pd.DataFrame:
        """
        Executes a query on the connected database and returns a pandas DataFrame
        Args:
            query: str: SQL query to execute
        Returns:
            pd.DataFrame: Query results
        """
        if self.connection is None:
            raise ConnectionError("Database not connected. Call connect() first.")
        
        return pd.read_sql_query(query, self.connection)

# Step 3: Define Abstract Factory
class DatabaseFactory(ABC):
    @abstractmethod
    def create_connector(self) -> Connector:
        """
        Abstract method for creating a database connector
        Returns:
            Connector: A concrete connector instance
        """
        pass

# Step 4: Implement Concrete Factory
class SQLiteFactory(DatabaseFactory):
    def __init__(self, db_name: str):
        self.db_name = db_name
    
    def create_connector(self) -> Connector:
        """
        Creates and returns a SQLite connector
        Returns:
            SQLite3Connector: A concrete SQLite connector instance
        """
        return SQLite3Connector(self.db_name)
    

# Example usage
if __name__ == "__main__":
    # Create factory
    sqlite_factory = SQLiteFactory("example.db")
    
    # Create connector using factory
    connector = sqlite_factory.create_connector()
    
    # Use connector
    connector.connect()
    results = connector.execute_query("SELECT * FROM some_table")

# Example usage
# Create factory
db_config =DataBaseConfig()
sqlite_factory = SQLiteFactory(db_config.db_path)

# Create connector using factory
connector = sqlite_factory.create_connector()

# Use connector
connector.connect()
results = connector.execute_query(db_config.sql_query)

In [82]:
results

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.0,C1305486145,181.0,0.0,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.0,C840083671,181.0,0.0,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.0,C776919290,0.0,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.0,C1881841831,0.0,0.0,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.0,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.0,C2080388513,0.0,0.0,1,0


In [68]:
class DataBaseConnection:

    def __init__ (self):
        """
        Initializes the connection with sqlite database
        """

        logger.info(f" Initializing connection with sqlite database")
        self.database_config = DataBaseConfig()
        try:
            self.connection = sqlite3.connect(self.database_config.db_path)
            logger.info(f"Successfully connected to the database {self.database_config.db_name}")
            pass
        except Exception as e:
            logger.error(f"Error connecting to data base : {e}")
            raise e

    def read_table_as_df(self):
        """
        Reads database table as a pandas data frame
        Args:
            query: str

        returns:
            pd.DataFrame
        """
        try:
            df = pd.read_sql_query(self.database_config.sql_query, con=self.connection)
            return df
        except Exception as e:
            logger.error(f" Error reading the database table as pandas DataFrame : {e}")
            raise e

In [69]:
db_conn = DataBaseConnection()

[2025-01-06 12:08:22,085, INFO, 1191946999,  Initializing connection with sqlite database]
[2025-01-06 12:08:22,088, INFO, 1191946999, Successfully connected to the database Database_Capstones]


In [70]:
df = db_conn.read_table_as_df()

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   step            object
 1   type            object
 2   amount          object
 3   nameOrig        object
 4   oldbalanceOrg   object
 5   newbalanceOrig  object
 6   nameDest        object
 7   oldbalanceDest  object
 8   newbalanceDest  object
 9   isFraud         object
 10  isFlaggedFraud  object
dtypes: object(11)
memory usage: 534.0+ MB
